In [1]:
system('conda env list | grep data730', intern = TRUE)

[1] "data730               *  /home/studio-lab-user/.conda/envs/data730"

In [2]:
library('conflicted')
library('dplyr')
library('tidyverse')
library('tidymodels')
library('recipes')
library('workflows')

Warning message:
“Your system is mis-configured: ‘/var/db/timezone/localtime’ is not a symlink”
Warning message:
“‘/var/db/timezone/localtime’ is not identical to any known timezone file”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.2.1     ✔ tune         1.2.1
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.1.0
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     



In [3]:
data_url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-11-23/episodes.csv"
dr_who_raw = read_csv(data_url, col_types = cols())
dr_who_raw |> head()

era,season_number,serial_title,story_number,episode_number,episode_title,type,first_aired,production_code,uk_viewers,rating,duration
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<date>,<chr>,<dbl>,<dbl>,<dbl>
revived,1,NA,157,1,Rose,episode,2005-03-26,1.1,10.81,76,45
revived,1,NA,158,2,The End of the World,episode,2005-04-02,1.2,7.97,76,44
revived,1,NA,159,3,The Unquiet Dead,episode,2005-04-09,1.3,8.86,80,44
revived,1,NA,160a,4,Aliens of London,episode,2005-04-16,1.4,7.63,82,45
revived,1,NA,160b,5,World War Three,episode,2005-04-23,1.5,7.98,81,42
revived,1,NA,161,6,Dalek,episode,2005-04-30,1.6,8.63,84,45


In [6]:
boots = bootstraps(dr_who_raw, times = 1000)

boots |> glimpse()

Rows: 1,000
Columns: 2
$ splits <list> [<boot_split[172 x 65 x 172 x 12]>], [<boot_split[172 x 55 x 1…
$ id     <chr> "Bootstrap0001", "Bootstrap0002", "Bootstrap0003", "Bootstrap00…


In [7]:
toss_fair_coin_n_times = function(n_tosses = 250) {
    result = ifelse(runif(n = n_tosses) < 0.5, 'heads', 'tails')
    result
}

repeat_the_experiment = function(n_iterations, n_tosses = 250) {
    n_heads_each_iteration = c()
    for (i in 1:n_iterations) {
        n_heads = sum(toss_fair_coin_n_times(n_tosses) == 'heads')
        n_heads_each_iteration = c(n_heads_each_iteration, n_heads)
    }
    n_heads_each_iteration
}

In [8]:
n_iterations = 10000
n_tosses = 250
observed_deviation = 139 - 125 #heads observed, expected value for a fair coin of 125

n_heads_each_experiment = repeat_the_experiment(n_iterations, n_tosses)

x = mean(abs(n_heads_each_experiment - 125) >= observed_deviation)
paste0('With a fair coin we could expect a deviation as large or larger than we observed ', x*100, '% of times')

[1] "With a fair coin we could expect a deviation as large or larger than we observed 8.61% of times"

# I think so